In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark Streaming Test")
    .getOrCreate()
)

spark

In [2]:
# Read input data

df_raw = spark.readStream.format("socket").option("host", "localhost").option("port", "9999").load()
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [3]:
# Split the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split("value", " "))

In [4]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")

In [5]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))       

In [6]:
# Write the output to console streaming

df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 1b268a4b-e5ab-4e2f-896d-a465fd78889f, runId = afc9a18c-18f0-41e6-a3bf-2f22d46d6324] terminated with exception: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'